Recently, I was working on image data and was running a for-loop to resize the images. This was embarrassingly parallel, but I wasn't familiar with running for-loops in parallel in python. This worksheet provides a template for for-loop parallelization. More information on parallel computing can be found [here](https://en.wikipedia.org/wiki/Parallel_computing).


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 
from tqdm import tqdm
from joblib import Parallel, delayed
from joblib import load, dump
from time import time

The data is from the competition titled [Planet: Understanding the Amazon from Space](https://www.kaggle.com/c/planet-understanding-the-amazon-from-space). The images used in this workbook are from the training dataset. I'm using the [cv2](http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_image_display/py_image_display.html) package to import and resize images. The [tqdm](https://pypi.python.org/pypi/tqdm) is a neat package that runs a progress bar as your loop runs. Need to use threading with the joblib package because the default *multiprocessing* option would not work on my windows machine.


In [15]:

def image_resize(file_hold):    
    img = cv2.imread('Z:\\Classes or School Work\\Eclipse Workspace\\Amazon image kaggle 201705\\Data\\train-jpg\\{}.jpg'.format(file_hold))
    return cv2.resize(img, (32, 32))
    
if __name__ == '__main__':
    
    df_train = pd.read_csv('Z:\\Classes or School Work\\Eclipse Workspace\\Amazon image kaggle 201705\\Data\\train_v2.csv')
    
    
    t0_4 = time()
    results_4 = Parallel(n_jobs=4, backend="threading")(delayed(image_resize)(f) for f in tqdm(df_train.values[:,0], miniters=1000)) 
    t1_4 = time()
    
    t0_8 = time()
    results_8 = Parallel(n_jobs=8, backend="threading")(delayed(image_resize)(f) for f in tqdm(df_train.values[:,0], miniters=1000)) 
    t1_8 = time()
    
    t0_0 = time()
    results_0 = []
    for f in tqdm(df_train.values[:,0], miniters=1000):
        results_0.append(image_resize(f))
    t1_0 = time()
    

100%|███████████████████████████████████| 40479/40479 [00:46<00:00, 862.62it/s]


In [16]:
print("The run time for the for-loop without running in parallel was %f" %(t1_0 - t0_0))
print("The run time for the for-loop with parallel on 4 cores was %f" %(t1_4 - t0_4))
print("The run time for the for-loop with parallel on 8 cores was %f" %(t1_8 - t0_8))

The run time for the for-loop without running in parallel was 46.963686
The run time for the for-loop with parallel on 4 cores was 14.210813
The run time for the for-loop with parallel on 8 cores was 11.293646


The 8-core for-loop doesn't decrease the time by half of the 4-core for-loop because there is a high cost of efficients for data i/o on the threads. This is a well-known phenomenon and means that more cores is not always better. 